In [ ]:
# Install all the dependencies, and access the Google Drive, where the data must be stored

!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install --upgrade accelerate

import os
import nltk
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration
from datasets import load_metric


nltk.download('punkt')

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# To fill with the path where the data is stored

os.chdir("gdrive/MyDrive/Colab Notebooks/Language")
!ls

In [ ]:
# Definition of the custom dataset class 

class SummarizationDataset(Dataset):

    def __init__(self, max_text_length=1024, max_summary_length=64, nb_inputs=None, model_checkpoint="t5-small"):

        self.max_text_length = max_text_length
        self.max_summary_length = max_summary_length
        self.nb_inputs = nb_inputs
        self.model_checkpoint = model_checkpoint
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.prefix = "summarize: "
        
    def process(self, filename):

        with open(filename, encoding='utf-8') as f:
            if self.nb_inputs is not None: df = pd.read_csv(f).head(self.nb_inputs) # Process only the nb first elements of the dataframe
            else: df = pd.read_csv(f) # Process the whole dataframe
        f.close()

        df.dropna(inplace=True) # Removes the NaN values from the dataframe
        self.total_lines = len(df)

        inputs = [self.prefix + text for text in df["Text"]] # Add the prefix to all the texts
        model_inputs = self.tokenizer(inputs, max_length=self.max_text_length, truncation=True, padding='max_length', return_tensors='pt') # The tokenizer truncates the long texts, and padds the short ones to the max_length
        labels = self.tokenizer(text_target=list(df["Summary"]), max_length=self.max_summary_length, truncation=True, padding='max_length', return_tensors='pt') # Same for the summaries
        model_inputs["labels"] = labels["input_ids"] # The ids of the labels are the labels of the inputs
        self.model_inputs = model_inputs

        print('Processed {} lines.'.format(self.total_lines))
    
    def __len__(self):
        return self.total_lines

    def __getitem__(self, index):
        return {'input_ids': self.model_inputs['input_ids'][index], 'attention_mask': self.model_inputs['attention_mask'][index], 'labels': self.model_inputs['labels'][index]}

In [ ]:
# Creation of the dataset from the file 'Reviews.csv'

dataset = SummarizationDataset(max_text_length=500, max_summary_length=40, nb_inputs=50000, model_checkpoint="t5-small")
dataset.process(filename='Reviews.csv')
tokenizer = dataset.tokenizer

In [ ]:
# Load the pretrained model: either t5-small if no model is stored, or 'model' if you have already started to fine-tune and saved the latest version
# And prepare for the training of the model
total_size = len(dataset)

model_checkpoint = "t5-small"
try:
    model = T5ForConditionalGeneration.from_pretrained("model") # In case a model has already been fine-tuned, and saved as "model"
    training_size = int(0.8 * total_size) 
    validation_size = int(0.1 * total_size)
    test_size = total_size - training_size - validation_size

except:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint) # In case no model has yet be fine-tuned
    training_size = 1 # In this case, we will simply evaluate the model, to have a reference for further training  
    validation_size = int(0.1 * total_size)
    test_size = total_size - training_size - validation_size

training_set, validation_set, test_set = random_split(dataset, [training_size, validation_size, test_size])

if torch.cuda.is_available(): 
    model.cuda()
  
batch_size = 4

args = Seq2SeqTrainingArguments(
    output_dir='/{}-finetuned-for-summarization'.format(model_checkpoint),
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

metric = load_metric("rouge")

def compute_metrics(to_evaluate): # Custom metric to evaluate the performance, we use the f1 score for the ROUGE-1, ROUGE-2 and ROUGE-L
    predictions, labels = to_evaluate
    predicted_summaries = tokenizer.batch_decode(predictions, skip_special_tokens=True) # The predicted summary, after decoding through the tokenizer
    true_summaries = tokenizer.batch_decode(labels, skip_special_tokens=True) # The original summary, after decoding through the tokenizer
    
    predicted_summaries = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in predicted_summaries] # Rouge expects a newline after each sentence
    true_summaries = ["\n".join(nltk.sent_tokenize(label.strip())) for label in true_summaries]
    
    result = metric.compute(predictions=predicted_summaries, references=true_summaries, use_stemmer=True)
    rouge1_result = result["rouge1"].mid.fmeasure * 100
    rouge2_result = result["rouge2"].mid.fmeasure * 100
    rougeL_result = result["rougeL"].mid.fmeasure * 100
    
    return {"rouge1": round(rouge1_result, 2), "rouge2": round(rouge2_result, 2), "rougeL": round(rougeL_result, 2)}
    
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=training_set,
    eval_dataset=validation_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Trains the model, and saves the updates as 'model'

trainer.train()
model.save_pretrained("model") 

In [ ]:
# Test the model on a few examples

model.cpu()

texts = [
    "summarize: I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of 'brown' chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.",
    "summarize: Great chips, great price.  Odds are that you will have them to yourself.  If you like your fish and chips soaked in malt vinegar, you will love these.  The best salt and vinegar chips I have ever had (and I love this flavor, mind you).  They are spectacular with deli sandwiches or on their own.  Since I have found that S&V is palatable to only the most intelligent of our species I know that my afternoon snack is all mine, unless of course I run into another heavy brained, hearty breathed like mind.  -Summary- If you like the taste of bitter sweet salty vinegar and a crispy chip to boot you wont pucker at the sight of these handily bagged morsels.  It is the first and only time I have committed to a whole case of chips and I will do it again.  Mmmmm... vinegar...",
    "summarize: These chips taste awesome. And unlike most other flavored chips, they actually make sure that plenty of the flavory salty goodness gets on each individual chip. Just don't pass gas near any pretty ladies after consumption. They'll totally know it was you.",
    "summarize: So I got this and tasted it strait out of the bottle, it tasted like smoky flavored milk - YUCK! I was depressed I was stuck with 4 bottles of this.  It sat on my shelf and I forgot about it. Last weekend I tasted a Zevia cream soda and was not pleased with the flavor, it tasted weird and smokey like the LorAnn oil I had gotten.  I tried to doctor up the soda with a little SF Vanilla Torani syrup and a tablespoon of heavy cream - IT TASTED AMAZING!!!  The odd smokiness was gone and it had a wonderful rich mouthfeel.<br /><br />So I wanted to see if this would help the LorAnn oil too.  I put a cup of whipping cream in a bowl and added a teaspoon of vanilla and tasted it, it tasted fine.  Then I added ONE DROP of the oil, it is very strong, and stirred it in - IT TASTED SO MUCH BETTER!  I whipped it up and put it on fruit and it was such a treat.<br /><br />Now I will always whip a drop of it with vanilla into desserts and whip cream.  It was such a happy accident to find out how to use it.  When added to vanilla it really adds a new depth of flavor and tastes so different than just out of the bottle or on its own in cream.<br /><br />It is VERY potent so only use a drop and increase after tasting, if you add too much it will ruin the recipe so use a light touch.",
]

summaries = [
    "Best sour cream & onion chip I've had", 
    "So Delicious...Yet my companions wont touch them.",
    "So much flavor your farts will smell like sweet onions",
    "Do not taste from bottle! Mix with vanilla for true flavor.",
]

for i in range(len(texts)):
    inputs = tokenizer(texts[i], return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_new_tokens=40, do_sample=False)
    print('Predicted summary: {}'.format(tokenizer.decode(outputs[0], skip_special_tokens=True)))
    print('Initial summary: {}\n'.format(summaries[i]))

Predicted summary: Delicious and a little too salty
Initial summary: Best sour cream & onion chip I've had

Predicted summary: The best chips ever
Initial summary: So Delicious...Yet my companions wont touch them.

Predicted summary: Awesome
Initial summary: So much flavor your farts will smell like sweet onions

Predicted summary: YUCK!
Initial summary: Do not taste from bottle! Mix with vanilla for true flavor.

